In [1]:
#!pip install requests pypdf 

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\accar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
import os
import re
import pandas as pd
import requests
from pypdf import PdfReader
from nltk.tokenize import sent_tokenize

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

C:\ProgramData\Anaconda3\envs\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
#DESCARGAR PDF SI NO EXISTE
# ============================================================
#def descargar_pdf(url, carpeta="pdfs"):
def descargar_pdf(url, carpeta):
    os.makedirs(carpeta, exist_ok=True)

    nombre = url.split("/")[-1].split("?")[0]
    if not nombre.lower().endswith(".pdf"):
        nombre += ".pdf"

    path_pdf = os.path.join(carpeta, nombre)

    if os.path.exists(path_pdf):
        return path_pdf

    try:
        r = requests.get(url, timeout=20)
        r.raise_for_status()
        with open(path_pdf, "wb") as f:
            f.write(r.content)
        return path_pdf

    except Exception as e:
        print(f"Error descargando {url}: {e}")
        return None


# EXTRAER TEXTO DE CADA PÁGINA DEL PDF
# ============================================================
def extraer_paginas_pdf(path_pdf):
    try:
        reader = PdfReader(path_pdf)
        return [(page.extract_text() or "") for page in reader.pages]
    except Exception as e:
        print(f"Error leyendo PDF {path_pdf}: {e}")
        return []


In [6]:
# EXTRAER SECCIÓN "INTRODUCCIÓN" EVITANDO ÍNDICE Y PORTADA
# ============================================================

# Regex robusta para detectar encabezado de sección
PATRON_INICIO = r"""
    (?:^|\n)                                # inicio de línea
    (?:\d+\s*\.?\s*|[IVX]+\.\s*)*           # opcional: numeraciones
    (Introducci[oó]n)                       # palabra clave
    \s*\n                                   # salto de línea = encabezado real
"""

# Regex genérica que detecta el siguiente título
PATRON_FIN = r"""
    (?:^|\n)
    (?:\d+\s*\.?\s*|[IVX]+\.\s*)+           # algo como "2." o "III."
    [A-ZÁÉÍÓÚÑ][^\n]{2,80}\n                # título siguiente
"""


def extraer_introduccion_regex_paginas(paginas, min_pagina=2, max_pagina=12):
    """
    páginas: lista de textos por página
    min_pagina: evita portada e índice
    max_pagina: evita procesar PDFs enormes
    """

    if len(paginas) == 0:
        return ""

    # Limitamos el rango útil
    paginas_utiles = paginas[min_pagina:max_pagina]
    texto = "\n".join(paginas_utiles)

    # Buscar encabezado real
    inicio = re.search(PATRON_INICIO, texto, flags=re.IGNORECASE | re.VERBOSE)
    if not inicio:
        return ""

    inicio_idx = inicio.start()

    # Buscar fin
    fin = re.search(PATRON_FIN, texto[inicio_idx:], flags=re.IGNORECASE | re.VERBOSE)
    if fin:
        fin_idx = inicio_idx + fin.start()
    else:
        fin_idx = len(texto)

    intro = texto[inicio_idx:fin_idx].strip()

    # Limpieza opcional con NLTK (mejora la coherencia)
    try:
        oraciones = sent_tokenize(intro, language="spanish")
        intro_limpia = " ".join(oraciones)
        return intro_limpia
    except:
        return intro


In [9]:
# PIPELINE COMPLETO: PDF + EXTRACCIÓN + VALIDACIÓN SEMÁNTICA
# ============================================================
import requests
import os
import numpy as np
import pandas as pd # Import pandas here for explicit use

def descargar_pdf(url, path_destino):
    try:
        r = requests.get(url, timeout=20)

        if r.status_code != 200:
            print(f"   → No se pudo descargar {url}: HTTP {r.status_code}")
            return False

        with open(path_destino, "wb") as f:
            f.write(r.content)

        # Validar encabezado PDF
        with open(path_destino, "rb") as f:
            if f.read(5) != b"%PDF-":
                print("   → Archivo descargado NO es PDF real (probable HTML).")
                os.remove(path_destino)
                return False

        return True

    except Exception as e:
        print(f"Error descargando {url}: {e}")
        return False

def procesar_archivo(csv_entrada, csv_salida, carpeta_pdf):
    df = pd.read_csv(csv_entrada, encoding='latin1', sep=";")

    if "pdf" not in df.columns:
        raise ValueError("El CSV debe contener una columna llamada 'pdf'.")

    # Create a copy to work on and avoid SettingWithCopyWarning
    df_processed = df.copy()
    
    # Initialize the 'introduccion' column for all rows
    df_processed["introduccion"] = ""

    # Crear carpeta si no existe
    os.makedirs(carpeta_pdf, exist_ok=True)

    for i, url in df_processed["pdf"].items():
        print(f"\nProcesando fila {i}:")
        print(f"URL original: {url}")
        print(f"Tipo de URL: {type(url)}")

        # Robustly check for NaN or empty string. pd.isna handles float('nan').
        if pd.isna(url) or (isinstance(url, str) and not url.strip()):
            print("   → URL no válida (NaN o vacía). Saltando.\n")
            df_processed.loc[i, "introduccion"] = "No existe pdf"
            continue  # Skip to the next iteration

        # Ensure URL is a string and clean it up
        url = str(url).strip()

        # Nombre del archivo PDF
        nombre_pdf_file = url.split("/")[-1].split("?")[0].strip()
        if not nombre_pdf_file.lower().endswith(".pdf"):
            nombre_pdf_file += ".pdf"
        
        path_pdf = os.path.join(carpeta_pdf, nombre_pdf_file)

        # If the PDF doesn't exist locally, download it
        if not os.path.exists(path_pdf):
            print(f"   → Descargando {url} a {path_pdf}...")
            exito = descargar_pdf(url, path_pdf)
        else:
            print(f"   → Archivo ya existe: {path_pdf}")
            exito = True  # It already exists

        if exito:
            df_processed.loc[i, "introduccion"] = "descargado"
            #Original commented out code for extraction and semantic validation
            try:
                # Extraer texto por páginas
                paginas = extraer_paginas_pdf(path_pdf)

                # Extraer sección introducción
                intro = extraer_introduccion_regex_paginas(paginas)

                # Validación semántica con embeddings
                if es_introduccion_valida(intro):
                    df_processed.loc[i, "introduccion"] = intro
                else:
                    print("   → Introducción NO válida.")
                    df_processed.loc[i, "introduccion"] = "Intro no válids"

            except Exception as e:
                print("Error leyendo PDF:", e)
                df_processed.loc[i, "introduccion"] = "No extrae Intro"
        else:
            print(f"   → NO se pudo descargar el PDF de {url}")
            df_processed.loc[i, "introduccion"] = "No descarga pdf"

    #df_processed.to_csv(csv_salida, index=False, encoding='latin1', sep=';')
    #print(f"\n>>> Archivo generado: {csv_salida}")
    return df_processed

In [ ]:
# EJECUTAR PIPELINE
# ============================================================

entrada="../datos/datos_carr_sel.csv"
salida = "../datos/datos_pdfs.csv"
carpeta_pdf ="../pdfs"
df_salida = procesar_archivo(entrada, salida, carpeta_pdf)
df_salida.to_csv(csv_salida, index=False, encoding='latin1', sep=';')
